# Mixture Density Network

This notebook trains a simple gaussian mixture density network from basic statistics of the predictive distributions coming from the component models.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append("../src")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import data
import utils
import losses

from functools import partial, reduce

from keras.layers import (Activation, Dense, Dropout, Embedding, Flatten, Merge, Reshape)
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import keras.backend as K
from sklearn.model_selection import train_test_split
from scipy.stats import norm

## Load data

In [ ]:
class Component:
    """
    Helper class for working with components
    """
    
    def __init__(self, name):
        self.name = name
        self.loader = data.ComponentDataLoader("../data", name)

In [ ]:
components = [Component(name) for name in [
    "ReichLab-KCDE",
    "Delphi-EmpiricalFuture",
    "ReichLab-SARIMA1",
    "CU-EAKFC"
]]
actual_dl = data.ActualDataLoader("../data")

### Working on week ahead predictions

We need to take the common row entries (common "epiweek", "region") for each data item, i.e. actual data and component data.

In [ ]:
REGION = None # Specify None for using all the data
WEEK_NUMBER = 1

y, Xs, yi = data.get_week_ahead_training_data(
    WEEK_NUMBER, REGION,
    actual_dl, [cmp.loader for cmp in components]
)

for idx, cmp in enumerate(components):
    cmp.data = Xs[idx]

### Extracting statistical features from the distributions

We are just using the mean and std of distributions and concatenating to a single vector as input to the model.

*X* refers to combined features from all the models.

*X_[model]* refers to full distributions from a particular model.

In [ ]:
X = utils.get_merged_features(
    Xs, 
    [utils.dist_mean, utils.dist_std]
)

### Split based on year
We take items before epiweek *201443* as train and rest as test

In [ ]:
train_indices = yi[:, 0] < 201443

## Model

The model is a simple mixture density network which returns a set of parameters which are then used in the loss function to get the negative log score for optimization

In [ ]:
def mdn(n_input, n_mix):
    """
    Return a mixture density model with given number of mixtures (gaussians)
    """
    
    model = Sequential()
    model.add(Dense(20, input_shape=(n_input,)))
    model.add(Activation("relu"))
    model.add(Dense(10))
#     model.add(Activation("relu"))
#     model.add(Dense(5))
    model.add(Activation("relu"))
    model.add(Dense(5))
    model.add(Activation("relu"))
    
    # Return 3 parameters, mu, sigma and mixture weight
    model.add(Dense(n_mix * 3))
    
    return model

In [ ]:
N_MIX = 1
loss_fn = partial(losses.mdn_loss, n_mix=N_MIX)
loss_fn.__name__ = "mdn_loss" # Keras needs a name for function

### Training

In [ ]:
# model generator
def gen_model():
    return mdn(X.shape[1], N_MIX)

def train_model(model, train_data, val_data):
    model.compile(optimizer="rmsprop", loss=loss_fn)

    early_stop = EarlyStopping(monitor="val_loss", patience=4, mode="auto")

    history = model.fit(train_data[0],
                        train_data[1],
                        batch_size=64, epochs=100,
                        verbose=1,
                        callbacks=[early_stop],
                        validation_data=val_data)
    return history

In [ ]:
[model, mean_losses, cv_metadata, final_history] = utils.cv_train(
    gen_model, train_model,
    X[train_indices], y[train_indices],
    k=10
)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])

### Evaluation

In [ ]:
regions = ["nat", *[f"hhs{i}" for i in range(1, 11)], None]

models = [*[cmp.name for cmp in components], "n-ensemble", "ave-ensemble", "prod-ensemble"]

eval_df = {model: [] for model in models}

for region in regions:
    if region is None:
        eval_indices = ~train_indices
    else:
        eval_indices = (~train_indices) & (yi[:, 1] == region)
        
    component_dists = [cmp.data[eval_indices] for cmp in components]
    n_dist = utils.mdn_params_to_dists(model.predict(X[eval_indices]))
    ave_dist = np.mean(component_dists, axis=0)
    
    prod_dist = reduce(np.multiply, component_dists)
    prod_dist /= prod_dist.sum(axis=0) + K.epsilon()

    dists = [
        *component_dists,
        n_dist,
        ave_dist,
        prod_dist
    ]
    y_one_hot = utils.wili_to_dists(y[eval_indices])
    
    for name, output in zip(models, dists):
        eval_df[name].append(K.categorical_crossentropy(output, y_one_hot).mean().eval())
eval_df = pd.DataFrame(eval_df)
eval_df.index = [*regions[:-1], "All"]
eval_df.to_csv(f"{WEEK_NUMBER}_eval.csv")

In [ ]:
eval_df

## Plot random predictions

In [ ]:
# Sample few of the examples
n_plots = 5
plot_indices = np.random.randint(0, y[~train_indices].shape[0], size=n_plots)

y_plot_out = y[~train_indices][plot_indices]

component_dists = [cmp.data[~train_indices][plot_indices] for cmp in components]

n_dist = utils.mdn_params_to_dists(model.predict(X[~train_indices][plot_indices]))
ave_dist = np.mean(component_dists, axis=0)
prod_dist = reduce(np.multiply, component_dists)
prod_dist /= prod_dist.sum(axis=0) + K.epsilon()

dists = [*component_dists, n_dist, ave_dist, prod_dist]

bins = np.linspace(0, 12.9, 130)
models = [*[cmp.name for cmp in components], "n-ensemble", "ave-ensemble", "prod-ensemble"]

for pidx in range(n_plots):
    plt.figure(figsize=(14, 6))
    for idx, model in enumerate(models):
        plt.plot(bins, dists[idx][pidx], label=model)
    
    # Plot actual line
    plt.axvline(x=y_plot_out[i])
    
    plt.legend()